In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
import time
try:                                    # Support Python 2 and 3
    from urllib import urlopen, build_opener          
except ImportError:                     
    from urllib.request import urlopen, build_opener  

    
# Helper function to open url
def open_url(stock, date):
    opener = build_opener()
    opener.addheaders = [('User-agent', 'Mozilla/5.0')]
    stocktable = opener.open('http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=' \
        + date + '&stockNo=' +str(stock))
    return stocktable.read()

#Extract Data from Months January to May for one stock
def extract_one_stock(stock):
    datelist = ['20180101', '20180201', '20180301', '20180401', '20180501']
    dflist = []
    for date in datelist:        
        page = open_url(stock, date)
        soup = BeautifulSoup(page, 'html.parser')
        table=soup.find('table')
        #Generate lists
        A=[]
        B=[]
        C=[]
        D=[]
        E=[]
        F=[]
        G=[]
        H=[]
        I=[]
        
        for row in table.find("tbody").find_all("tr"):
            cells = row.find_all("td")
            if len(cells) == 9:
                A.append(cells[0].find(text=True))
                B.append(cells[1].find(text=True))
                C.append(cells[2].find(text=True))
                D.append(cells[3].find(text=True))
                E.append(cells[4].find(text=True))
                F.append(cells[5].find(text=True))
                G.append(cells[6].find(text=True))
                H.append(cells[7].find(text=True))
                I.append(cells[8].find(text=True))
            
        df = pd.DataFrame(A,columns=['日期'])
        df['成交股數']=B
        df['成交金額']=C
        df['開盤價']=D
        df['最高價']=E
        df['最低價']=F
        df['收盤價']=G
        df['漲跌價差']=H
        df['成交筆數']=I
        
        dflist.append(df)
        
    
    # putting all monthly dataframes together for the same stock
    final_df = pd.DataFrame()
    for i in range(0, len(dflist)):
        final_df = final_df.append(dflist[i])
      
    writer = pd.ExcelWriter(str(stock)+'.xlsx', engine='xlsxwriter')   
    final_df.to_excel(writer,sheet_name=str(stock),startrow=0 , startcol=0)   
    writer.save()
    
# Extract multiple stocks
def extract_mult_stocks(stocks):
    for stock in stocks:
        extract_one_stock(stock)
        wait = random.randint(10,20)
        time.sleep(wait)

# Extract Data for all 52 stocks 
stocks = [1101, 1102, 1216, 1301, 1303, 1326, 1402, 2002, 2105,
          2227, 2301, 2303, 2308, 2317, 2327, 2330, 2352, 2354, 2357, 2382, 2395, 2408,
          2409, 2412, 2454, 2474, 2609, 2610, 2633, 2801, 2823, 2880, 2881, 2882, 2883,
          2884, 2885, 2886, 2887, 2890, 2891, 2892, 2912, 3008, 3045, 3481, 4904, 4938, 5871,
          5880, 6505, 9904]

extract_mult_stocks(stocks)
        
        

           日期        成交股數           成交金額    開盤價    最高價    最低價    收盤價   漲跌價差  \
0   107/01/02  13,698,944    499,370,945  36.45  36.60  36.05  36.55  +0.10   
1   107/01/03  19,959,825    737,353,608  36.50  37.35  36.25  37.35  +0.80   
2   107/01/04  14,927,721    555,392,977  37.35  37.45  36.75  37.20  -0.15   
3   107/01/05  35,536,815  1,325,926,954  37.50  37.50  37.05  37.30  +0.10   
4   107/01/08  22,660,681    858,574,153  37.40  38.30  37.20  38.20  +0.90   
5   107/01/09  19,053,008    728,607,834  38.10  38.55  37.90  38.30  +0.10   
6   107/01/10  11,674,625    447,720,584  38.30  38.50  38.00  38.45  +0.15   
7   107/01/11  35,062,055  1,322,632,044  38.30  38.35  37.45  37.50  -0.95   
8   107/01/12  18,804,747    707,643,024  37.75  37.95  37.35  37.60  +0.10   
9   107/01/15   9,637,157    363,246,276  37.65  37.85  37.55  37.85  +0.25   
10  107/01/16   9,338,163    354,474,155  37.95  38.10  37.70  38.10  +0.25   
11  107/01/17  12,716,828    487,501,985  38.00  38.

KeyboardInterrupt: 